## Summary

Repo: https://github.com/pgahq/instructor-groq-openai-llm-examples

This notebook shows how to use Instructor to extract structured info from unstructured text where the responses are constrained by an enumerated list. Instructor handles [Enum and Literal](https://jxnl.github.io/instructor/concepts/enums/) differently. Literal seems simpler.

Note: this notebook assumes you're using Google Colab. You can safely edit / play here. Or go to `File` -> `Save a copy in Google Drive` to make your own version.

In [1]:
%%capture
!pip install instructor groq openai

On the left, click the key and set two secrets with your keys. Be sure to enable "Notebook access" for them. This is how Google Colab works...you're not sharing your keys with anyone.

OPENAI_API_KEY

GROQ_API_KEY

In [2]:
import instructor
import openai
import groq
from pydantic import BaseModel, Field
from typing import Optional, List, Literal
import os

try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = '' or userdata.get('OPENAI_API_KEY') # or put your key in the '' on this line
    os.environ['GROQ_API_KEY'] = '' or userdata.get('GROQ_API_KEY')
except Exception as e:
    # print(e)
    pass


In [7]:
inference_provider = "openai"   # "openai" or "groq"
client = instructor.from_openai(openai.OpenAI()) if inference_provider == "openai" else instructor.from_groq(groq.Groq())

class Review(BaseModel):
    business_type: Literal["dining establishment", "service business", "hotel", "other"] = Field(description="Type of business.")
    sentiment: Literal["good vibes", "ok-ish", "not incredible"] = Field(description="Sentiment of the review.")

response = client.chat.completions.create(
    model="llama3-70b-8192" if inference_provider == "groq" else "gpt-4o",
    response_model=Review, # this is Instructor at work!
    temperature=0.0,
    messages=[{"role": "user", "content": "This place has amazing biscuits!"}]    # intentionally vague so the LLM will need to infer the business type
    # messages=[{"role": "user", "content": "This place has amazing showers!"}] 
    # messages=[{"role": "user", "content": "This place has amazing appointment scheduling / reminders!"}] 
    )

print(response.model_dump_json(indent=4))


{
    "business_type": "dining establishment",
    "sentiment": "good vibes"
}
